## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [2]:
spark

In [3]:
sqlContext

### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [4]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [5]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [6]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
dbs.toPandas()

,databaseName
0,country_club
1,default


# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [7]:
# File location and type
file_location_bookings = "./Bookings.csv"
file_location_facilities = "./Facilities.csv"
file_location_members = "./Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [8]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [9]:
permanent_table_name_bookings = "country_club.Bookings1"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities1"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members1"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [10]:
run_sql('use country_club')
run_sql('REFRESH table bookings1')
run_sql('REFRESH table facilities1')
run_sql('REFRESH table members1')
tbls = run_sql('show tables')
tbls.toPandas()

,database,tableName,isTemporary
0,country_club,bookings1,False
1,country_club,facilities1,False
2,country_club,members1,False


# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [11]:
result = run_sql('''
                    SELECT * 
                    FROM bookings1
                   
                 ''')
result.toPandas()


,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1
5,5,8,1,2012-07-03 15:00:00,1
6,6,0,2,2012-07-04 09:00:00,3
7,7,0,2,2012-07-04 15:00:00,3
8,8,4,3,2012-07-04 13:30:00,2
9,9,4,0,2012-07-04 15:00:00,2


# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [12]:

result = run_sql('''
                  select 
                   name
                          
                  from facilities1
                  where membercost <> 0 
                  
                  
                 ''')
result.toPandas()


,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [13]:
result = run_sql('''
                select count(*)
                from facilities1
                where membercost = 0 
                 ''')
result.toPandas()

,count(1)
0,4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [14]:
result = run_sql('''
                select 
                facid
                ,name
                ,membercost
                ,monthlymaintenance
                
                from facilities1
                where membercost < 0.2 *monthlymaintenance 
                and membercost <> 0
                ''')
result.toPandas()

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [15]:
result = run_sql('''
                select *
                 from facilities1
                 where facid in (1,5)
                ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [16]:
result = run_sql('''
                select name
                , case when monthlymaintenance > 100 then 'expensive' 
                else 'cheap' end as lable
               
                from facilities1 
                 ''')
result.toPandas()

,name,lable
0,Tennis Court 1,expensive
1,Tennis Court 2,expensive
2,Badminton Court,cheap
3,Table Tennis,cheap
4,Massage Room 1,expensive
5,Massage Room 2,expensive
6,Squash Court,cheap
7,Snooker Table,cheap
8,Pool Table,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [17]:
result = run_sql('''
                select 
                surname
                ,firstname
                from members1
                WHERE
                memid 
                = 
                (
                select max(memid)
                from members1
                )
                
                 ''')
result.toPandas()

,surname,firstname
0,Smith,Darren


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [18]:
result = run_sql('''
                 
                  select 
                  
                  concat(b.firstname,' ' ,b.surname) as member_name
                  ,c.name
                  
                  from bookings1 a
                  left join
                  members1 b
                  on a.memid = b.memid
                  left join
                  facilities1 c
                  on a.facid = c.facid
                  where c.name like ('%Tennis Court%')
                  and b.firstname <> 'GUEST'
                  group by 
                  
                  member_name
                  ,c.name
                  order by member_name
                  
                 ''')
result.toPandas()

,member_name,name
0,Anne Baker,Tennis Court 1
1,Anne Baker,Tennis Court 2
2,Burton Tracy,Tennis Court 2
3,Burton Tracy,Tennis Court 1
4,Charles Owen,Tennis Court 2
5,Charles Owen,Tennis Court 1
6,Darren Smith,Tennis Court 2
7,David Farrell,Tennis Court 2
8,David Farrell,Tennis Court 1
9,David Jones,Tennis Court 2


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [33]:
result = run_sql('''

                  select 
                 *
                  from
                 facilities1
                 limit 3
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50


In [36]:
result = run_sql('''

                  select 
                  name
                  ,member_name
                  ,cost
                  from
                  (
                  select  
                   a.*
                  ,concat(b.firstname,' ' ,b.surname) as member_name
                  ,c.membercost
                  ,c.guestcost
                  ,c.name
                  ,case when a.memid = 0 then slots * c.guestcost else slots * c.membercost end as cost
                  from bookings1 a
                  left join
                  members1 b
                  on a.memid = b.memid
                  left join
                  facilities1 c
                  on a.facid = c.facid 
                  
                  where a.starttime like ('%2012-09-14%')
                  order by member_name
                  )
                  where cost > 30
                  order by cost desc
                 ''')
result.toPandas()

,name,member_name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [37]:
result = run_sql('''

                  select 
                  name
                  ,member_name
                  ,cost
                  from
                  (
                  select  
                   a.*
                  ,concat(b.firstname,' ' ,b.surname) as member_name
                  ,c.membercost
                  ,c.guestcost
                  ,c.name
                  ,case when a.memid = 0 then slots * c.guestcost else slots * c.membercost end as cost
                  from bookings1 a
                  left join
                  members1 b
                  on a.memid = b.memid
                  left join
                  facilities1 c
                  on a.facid = c.facid 
                  
                  where a.starttime like ('%2012-09-14%')
                  order by member_name
                  )
                  where cost > 30
                  order by cost desc
                 ''')
result.toPandas()

,name,member_name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [41]:
result = run_sql('''
                   select 
                  name, sum(cost) total_revenue
                
                  from
                  (
                  select  
                   a.*
                  ,concat(b.firstname,' ' ,b.surname) as member_name
                  ,c.membercost
                  ,c.guestcost
                  ,c.name
                  ,case when a.memid = 0 then slots * c.guestcost else slots * c.membercost end as cost
                  from bookings1 a
                  left join
                  members1 b
                  on a.memid = b.memid
                  left join
                  facilities1 c
                  on a.facid = c.facid 
                
                  order by member_name
                  )
                  group by name
                  order by total_revenue desc
            
                 ''')
result.toPandas()

,name,total_revenue
0,Massage Room 1,50351.6
1,Massage Room 2,14454.6
2,Tennis Court 2,14310.0
3,Tennis Court 1,13860.0
4,Squash Court,13468.0
5,Badminton Court,1906.5
6,Pool Table,270.0
7,Snooker Table,240.0
8,Table Tennis,180.0
